In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import random
from utils import plot_samples_with_labels, classify, plot_conf_matrix

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Set the seed for reproducibility
random.seed(42)

### Load and explore data

In [1]:
# Download the MNIST dataset (handwritten digit images with labels)
mnist = keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

NameError: name 'keras' is not defined

In [ ]:
plot_samples_with_labels(training_images, training_labels, num_samples = 25, cmap = plt.cm.binary, randomize= False)

#### Data preparation

In [ ]:
training_images = training_images / 255.0
test_images = test_images / 255.0

# Reshape the data to include a channel dimension
training_images = training_images.reshape(training_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

### Build the model

In [ ]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential

model = Sequential([
    Flatten(input_shape=(28, 28, 1)),  # 1 = grayscale
    Dense(units=50, activation='relu', input_shape=(28, 28, 1)),
    Dense(units=50, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

### Train the model

In [ ]:
BATCH_SIZE = 32
num_training_images = training_images.shape[0]

history = model.fit(
    training_images, training_labels,
    epochs=10,
    batch_size=BATCH_SIZE,
    validation_data=(training_images, training_labels)
)

In [ ]:
model.save('models/digit_non_CNN.h5')

### Using the model to classify new images

In [ ]:
from tensorflow.keras.models import load_model

model_non_cnn = load_model('models/digit_non_CNN.h5')

In [ ]:
def classify(test_images, model):
    """
    Predicts the class labels for a batch of test images using a trained model.

    Args:
        test_images (numpy.ndarray): Preprocessed test images ready for prediction.
        model (tensorflow.keras.Model): A trained model to classify the images.

    Returns:
        numpy.ndarray: An array of predicted class labels.
    """
    predictions = model.predict(test_images)    # Predict probabilities
    predicted_class = np.argmax(predictions, axis=-1)    # Get class with highest probability
    return predicted_class


In [ ]:
predicted_labels = classify(test_images, model_non_cnn)
predicted_labels = np.array(predicted_labels)

In [ ]:
plot_samples_with_labels(test_images, test_labels, predicted_labels, num_samples = 10, randomize= True)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate metrics
print("Accuracy:", accuracy_score(test_labels, predicted_labels))
print("\nClassification Report:\n")
print(classification_report(test_labels, predicted_labels))

In [ ]:
# Generate confusion matrix
plot_conf_matrix(test_labels, predicted_labels)

In [ ]:
# Visualize some misclassified images

misclassified_indices = (test_labels != predicted_labels)
misclassified_images = test_images[misclassified_indices]
misclassified_true_labels = test_labels[misclassified_indices]
misclassified_predicted_labels = predicted_labels [misclassified_indices]

if len(misclassified_images) > 0:
    plot_samples_with_labels(misclassified_images, misclassified_true_labels, misclassified_predicted_labels, num_samples = 10, randomize= True)
else:
    print("No misclassified images found in the selected batch.")


Qué dificultades encuentra nuestro modelo?

Ideas para solucionarlas?
